In [8]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import plotly.express as px

In [9]:
import sys
from io import StringIO
import numpy as np
import pandas as pd

# TODO: 
# - Figure out threshold
# - Figure out order of magnitude for generalization

### INP FILES: 
def extract_inp_table(inp_contents: str, start_keyword: str, end_keyword: str):
    start_index = inp_contents.find(start_keyword)
    end_index = inp_contents.find(end_keyword)

    start_index += len(start_keyword) + 1
    extracted_text = inp_contents[start_index:end_index]

    return extracted_text

def get_node_df(inp_contents: str): # from inp file
    start_keyword = "*NODE"
    end_keyword = "**HWCOLOR COMP"
    table = extract_inp_table(inp_contents, start_keyword, end_keyword)

    node_table_df = pd.read_csv(
        StringIO(table),
        sep=r'[,\s]+',
        header=None,
        names=[
            'node_no',
            'x',
            'y',
            'z'
        ],
        engine='python'
    )

    return node_table_df

### DAT FILES:
def extract_table(contents: str, keyword: str, delim: str ='\n\n\n') -> str:
    """
    Extracts strings consisting of only table values specific for .dat files
    """
    # Find keyword index
    start_index = contents.find(keyword)
    if start_index == -1:
        raise ValueError(f"Keyword '{keyword}' not found in file.")
    
    start_index += len(keyword)
    
    # Start remaining text after keyword
    remaining_text = contents[start_index:]
    
    # Find 2 occurrences of delim
    search_start = 0
    
    count = 0
    while count != 2:
        pos = remaining_text.find(delim, search_start)
        if pos == -1:
            break
        if count == 0:
            table_start_index = pos + 3
        elif count == 1:
            table_end_index = pos
            
        count += 1
        search_start = pos +3
    
    try:
        extracted_text = remaining_text[table_start_index:table_end_index]
    except:
        extracted_text = remaining_text[table_start_index:]
    
    return extracted_text

def get_mode_table_df(contents: str):
    keyword = 'E I G E N V A L U E    O U T P U T'
    table = extract_table(contents, keyword)

    mode_table_df = pd.read_csv(
        StringIO(table),
        sep=r'\s+',
        header=None,
        names=[
            'mode_no',
            'eigenvalue',
            'freq (rad/time)',
            'freq (cycles/time)',
            'generalized mass',
            'composite model damping'
        ]
    )

    mode_table_df = mode_table_df[['mode_no', 'freq (cycles/time)']].copy()
    mode_table_df.rename(columns = {'freq (cycles/time)': 'freq'}, inplace=True)

    return mode_table_df

def get_mode_df(contents, mode_number):
    num_str = str(mode_number)
    keyword = f"E I G E N V A L U E    N U M B E R{num_str.rjust(6, ' ')}"
    table_content_start = contents.find(keyword)
    table_content = contents[table_content_start + len(keyword):]

    table = extract_table(table_content, 'U3', delim='\n  \n')

    max_index = table.find('MAX') 
    if max_index == -1:
        raise Exception("Incorrect file format")
    table = table[:max_index].rstrip()

    # print(table)
    mode_df = pd.read_csv(
        StringIO(table),
        sep=r'\s+',
        header=None,
        names=[
            'node_no',
            'U1',
            'U2',
            'U3'
        ]
    )

    return mode_df    

def plot_sum_graph(mode_no):
    node_df = get_node_df(inp_contents)
    mode_df = get_mode_df(contents, mode_no)
    mode_df['resultant'] = np.sqrt(mode_df['U1']**2 + mode_df['U2']**2 + mode_df['U3']**2)
    sum_df = mode_df[['node_no','resultant']] # node and sum
    merged_df = pd.merge(node_df, sum_df, on='node_no', how='inner')
    df = merged_df
    display(df)
    
    custom_colorscale = [
        [0.0, '#00008B'],  # dark blue
        [0.5, '#FFFF00'],  # yellow
        [1.0, '#FF0000'],  # red
    ]

    min_val = min(df['x'].min(), df['y'].min(), df['z'].min())
    max_val = max(df['x'].max(), df['y'].max(), df['z'].max())
    
    fig = px.scatter_3d(
        df,
        x='x',
        y='y',
        z='z',
        color='resultant',
        color_continuous_scale=custom_colorscale,
        size_max=10,
        hover_data=['node_no', 'resultant']
    )
    
    fig.update_layout(
        title=f"Mode number ({mode_no})",
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(range=[min_val, max_val]),
            yaxis=dict(range=[min_val, max_val]),
            zaxis=dict(range=[min_val, max_val]),
        )
    )
    
    fig.show(renderer='browser')

    return df

def plot_graph(df):
    # Calculate the combined min and max over all three axes
    min_val = min(df['x'].min(), df['y'].min(), df['z'].min())
    max_val = max(df['x'].max(), df['y'].max(), df['z'].max())
    custom_colorscale = [
        [0.0, '#00008B'],  # dark blue
        [0.5, '#FFFF00'],  # yellow
        [1.0, '#FF0000'],  # red
    ]   
    fig = px.scatter_3d(
        df,
        x='x',
        y='y',
        z='z',
        size_max=2,
        color='x',
        color_continuous_scale=custom_colorscale,
        hover_data=['node_no']
    )


    
    fig.update_layout(
        title='Brake Rotor Full Layout',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            xaxis=dict(range=[min_val, max_val]),
            yaxis=dict(range=[min_val, max_val]),
            zaxis=dict(range=[min_val, max_val]),
        )
    )
    
    fig.show(renderer='browser')

In [10]:
def contains_node(mode_df, threshold=7, lower_p_thres=0):
    mode_df['resultant'] = np.sqrt(mode_df['U1']**2 + mode_df['U2']**2 + mode_df['U3']**2)
    zero_proportion = (mode_df['resultant'] < threshold).sum()/len(mode_df)
    return zero_proportion > lower_p_thres

In [11]:
dat_file = "../data/C346RS_10Jun/C346RS_frnt_rotor_modal_separation_10Jun25.dat"
inp_file = "../data/C346RS_10Jun/C346RS_frnt_rotor_modal_separation_10Jun25.inp"

In [12]:
# dat_file = 'data/V801_17Jun/V801_frnt_rotor_modal_separation_17Jun25 (1).dat'
# inp_file = 'data/V801_17Jun/V801_frnt_rotor_modal_separation_17Jun25.inp'

In [13]:
with open(dat_file, "r") as file:
    contents = file.read()

In [14]:
with open(inp_file, "r") as inp_file:
    inp_contents = inp_file.read()

In [15]:
outplane_modes = [29, 36, 45, 48, 65, 71, 94, 103]
inplane_modes = [32, 33, 46, 47, 68, 69, 99, 100]

In [16]:
node_df = get_node_df(inp_contents)

In [17]:
mode_table_df = get_mode_table_df(contents)

In [22]:
mode_dfs = mode_table_df

In [25]:
mode_dfs.index

,mode_no,freq
0,1,325.94
1,2,325.94
2,3,489.90
3,4,582.06
4,5,582.06
...,...,...
126,127,15824.00
127,128,15824.00
128,129,15878.00
129,130,15957.00


In [18]:
# plot_graph(node_df)
# plot_sum_graph(1)

In [24]:
inrange_modes

[]

In [29]:
len(freq)

131

In [30]:
freq = df['freq']
new_freq = []
new_modes = []

for i in range(len(freq)-1):
    j = 1
    while i+j < len(freq)-1 and freq[i+j] - freq[i] <= 300:
        new_freq.append((freq[i].item(), freq[i+j].item()))
        new_modes.append((i+1, i+j+1))
        j += 1
# think of instances where 1 freq has >= 2 modes

In [32]:
new_modes

[(1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (3, 4),
 (3, 5),
 (4, 5),
 (6, 7),
 (6, 8),
 (7, 8),
 (7, 9),
 (7, 10),
 (8, 9),
 (8, 10),
 (9, 10),
 (11, 12),
 (11, 13),
 (12, 13),
 (13, 14),
 (13, 15),
 (13, 16),
 (13, 17),
 (14, 15),
 (14, 16),
 (14, 17),
 (15, 16),
 (15, 17),
 (16, 17),
 (18, 19),
 (20, 21),
 (20, 22),
 (20, 23),
 (21, 22),
 (21, 23),
 (22, 23),
 (22, 24),
 (23, 24),
 (25, 26),
 (25, 27),
 (25, 28),
 (25, 29),
 (26, 27),
 (26, 28),
 (26, 29),
 (27, 28),
 (27, 29),
 (28, 29),
 (30, 31),
 (30, 32),
 (30, 33),
 (31, 32),
 (31, 33),
 (32, 33),
 (32, 34),
 (32, 35),
 (33, 34),
 (33, 35),
 (34, 35),
 (36, 37),
 (38, 39),
 (40, 41),
 (40, 42),
 (40, 43),
 (40, 44),
 (40, 45),
 (41, 42),
 (41, 43),
 (41, 44),
 (41, 45),
 (42, 43),
 (42, 44),
 (42, 45),
 (43, 44),
 (43, 45),
 (44, 45),
 (46, 47),
 (48, 49),
 (50, 51),
 (50, 52),
 (50, 53),
 (51, 52),
 (51, 53),
 (52, 53),
 (54, 55),
 (56, 57),
 (58, 59),
 (58, 60),
 (58, 61),
 (59, 60),
 (59, 61),
 (60, 61),

In [ ]:
# instances where 1 freq has >= 2 modes

In [ ]:
node_df = get_node_df(inp_contents)
def get_mode_node_df(mode_no):
    mode_df = get_mode_df(contents, mode_no)
    merged_df = pd.merge(node_df, mode_df, on='node_no', how='inner')
    df = merged_df
    return df

In [ ]:
df = get_mode_node_df(1)

In [ ]:
def get_vectors(mode_no):
    df = get_mode_node_df(mode_no)
    df['R'] = np.hypot(df['x'], df['z']) # sqrt(x^2 + z^2)
    R_safe = df['R'].replace(0, np.nan)
    
    # Unit r = (x/R, 0, z/R)
    df['r_hat_x'] = df['x'] / R_safe
    df['r_hat_z'] = df['z'] / R_safe
    
    # Unit n = (0, 1, 0)
    
    # Unit t = unit n x unit r =  (z/R, 0, -x/R)
    df['t_hat_x'] = df['r_hat_z']
    df['t_hat_z'] = -df['r_hat_x']
    
    # Tangential component = U dot t_hat
    U_t = np.dot(df['U1'], df['t_hat_x']) + np.dot(df['U3'], df['t_hat_z'])
    # Radial component = U dot r_hat
    U_r = np.dot(df['U1'], df['r_hat_x']) + np.dot(df['U3'], df['r_hat_z'])
    # Normal component = U dot n_hat
    U_n = np.sum(df['U2'])


    # Spherical coordinates
    df['U_rho'] = np.sqrt(df['U1']**2 + df['U2']**2 + df['U3']**2)
    U_rho = df['U_rho'].sum()
    U_theta = np.arctan(df['U3'] / df['U1']).sum()
    U_phi = np.arctan(df['U2'] / df['U_rho']).sum()

    return U_rho, U_theta, U_phi, U_t, U_r, U_n

In [ ]:
for i in range(1, 132):
    U_rho, U_theta, U_phi, U_t, U_r, U_n = get_vectors(i)

    print(U_rho)